In [5]:
import csv
import math
import json
from math import sqrt

# читаем файлы с данными
def csv_reader(path):
    with open(path) as csv_data:
        data = list(csv.reader(csv_data, delimiter=',', skipinitialspace=True))
    return data

#ищем среднюю оценку пользователя
def avg(data):
    rated = list(filter(lambda x: x != -1, data))
    avg=sum(rated)/len(rated)
    return avg

#упорядоченная метрика сходства
def similarity(userU, data):
    userU_ratings = data[userU]
    
    sim={}
    for userV in range(1,len(data)):
        if userV == userU:
            continue   
        userV_ratings = data[userV]
        UV, userU_norm, userV_norm = 0, 0, 0
        
        for i in range(len(userU_ratings)):
            if userU_ratings[i] == -1 or userV_ratings[i] == -1:
                continue
            #числитель
            UV += userU_ratings[i] * userV_ratings[i]
            #знаменатели
            userU_norm += userU_ratings[i] * userU_ratings[i]
            userV_norm += userV_ratings[i] * userV_ratings[i]
            sim[userV] = round(UV / (sqrt(userU_norm)*sqrt(userV_norm)), 3)
    sim = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    return sim

#рассчет оценки
def expectedRating(userU, data, k):
    avgU = avg(data[userU])
    expectedRatings = {}
    userU_ratings = data[userU]
    simList=similarity(userU, data); 
    #для каждого фильма, который еще не посмотрел пользователь
    for movie in range(len(userU_ratings)):
        count, num, denom =0,0,0
        if userU_ratings[movie] == -1:
            #берем первых k похожих
            for i in range(0,k):
                userV = simList[i] #пара (userId, sim)
                if data[userV[0]][movie] != -1:
                    simVU = userV[1]
                    rV = avg(data[userV[0]])
                    rVI = data[userV[0]][movie]
                    num += simVU * (rVI - rV)
                    denom += abs(simVU)
            rating = round(avgU + num/denom, 3)
            expectedRatings["movie " + str(movie+1)] = rating
    return expectedRatings


In [6]:
"""
проверяем каждый фильм в базе 
берем первых k похожих на нашего юзера и проверяем: 
    если пользователь смотрел фильм дома в выходные и оценил, 
    то добавляем его оценку в список рейтинга фильма, 
    а сам фильм в список возможных рекомендаций 
считаем среднюю оценку для каждого из этих фильмов 
затем смотрим какие из них искомый пользователь еще не смотрел 
и выбирам тот, который имеет наивысший ожидаемый рейтинг

"""

def weekendFilms(context_day, context_place, data, userID):
    simList = similarity(userID, data) # отсчитывает ID с 0
    films = {}
    for movie in range(len(data[userID])):
        ratings = []
        for i in range(0,k):
            userV = simList[i] #пара (userId, sim)
            if data[userV[0]-1][movie]!=-1 and context_place[userV[0]][movie+1] == "h" and context_day[userV[0]][movie+1] == "Sun" or context_day[userV[0]][movie+1] == "Sat":
                #print("mov: ", movie)
                #print("user ", userV[0], "rating: ", data[userV[0]-1][movie])
                #добавляем 
                ratings.append(data[userV[0]-1][movie])
        if len(ratings) > 0:
            films[movie] = round(sum(map(int, ratings))/len(ratings), 3)
    return films

def getResult(context_day, context_place, data, userID):
    wkndFilms = weekendFilms(context_day, context_place, data, userID)
    expRating = expectedRating(userID, data, k)
    
    recMovie = {}
    for movie in wkndFilms:
        if data[userID][movie] == -1:
            recMovie["movie " + str(movie+1)] = expRating["movie " + str(movie+1)]
    recMovie = sorted(recMovie.items(), key=lambda x: x[1], reverse=True)
    result = { 
        'user': userID+1,
        '1': expRating,
        '2': recMovie[0]
    }
    return result

In [7]:
context_day = []
context_day = csv_reader("context_day.csv")
context_place = []
context_place = csv_reader("context_place.csv")
data = []
for row in csv_reader("data.csv")[1:]:
    data.append(list(map(int, row[1:])))

k = 4
user =7  #userID нормального человека
userID = user - 1  #userID программиста

result = getResult(context_day, context_place, data, userID)
print(result)

json_file = open("output.json",'w')
json_file.write(json.dumps(result))
json_file.close()

{'user': 7, '1': {'movie 10': 2.14, 'movie 12': 3.395, 'movie 17': 1.288, 'movie 18': 3.214, 'movie 26': 4.639}, '2': ('movie 12', 3.395)}
